### Agents

##### Question 1 - Adaptive Writing Format

In [57]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.agents import tool, create_react_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain import hub
import os
import re

In [58]:
llm = ChatGoogleGenerativeAI(model='gemini-pro', convert_system_message_to_human=True)
chat = ChatOpenAI(openai_api_key = os.getenv("OPENAI_API_KEY_PERSONAL"))

In [59]:
user_input = str(input("Please enter the text you want to reformat: "))

sysTmp = "You are an AI assistant that helps the user to tell them what style their content belong to (for example : Formal, Informal, Conversational, etc). You answer in only one word."
sysMsgPmtTmp = SystemMessagePromptTemplate.from_template(sysTmp)

humTmp = "What is the style of the following text. \n\n {user_input}"
HumMsgPmtTmp = HumanMessagePromptTemplate.from_template(humTmp)

chatMsgPmtTmp = ChatPromptTemplate.from_messages([sysMsgPmtTmp, HumMsgPmtTmp])
finalPrompt = chatMsgPmtTmp.format_prompt(user_input=user_input) 


In [60]:
result = llm.invoke(finalPrompt)
print(result.content)

Formal


In [61]:
tools = []

@tool
def remove_citations(text):
    """
    Use this tool when you want to remove the citations from any text.
    """
    cleaned_text = re.sub(r'\[[0-9]+\]', '', text)
    return cleaned_text

@tool
def desired_output(user_input):
    """
    Useful when user asks to convert the text format or style to another.
    """
    sysTmp = "You are an AI assistant that helps the user to convert one style text to another."
    sysMsgPmtTmp = SystemMessagePromptTemplate.from_template(sysTmp)
    humTmp = "Convert the following text as per the instruction \n\n {user_input}"
    HumMsgPmtTmp = HumanMessagePromptTemplate.from_template(humTmp)
    chatMsgPmtTmp = ChatPromptTemplate.from_messages([sysMsgPmtTmp, HumMsgPmtTmp])
    desiredOutput = chatMsgPmtTmp.format_prompt(user_input=user_input) 
    return desiredOutput

tools = tools + [remove_citations, desired_output]

In [62]:
flag=1
while(flag):
    desired_format = user_input + input("Please specify your desired output format: ") 
    agentPrompt = hub.pull('hwchase17/react')
    agent = create_react_agent(llm=chat, tools=tools, prompt=agentPrompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools ,verbose=True, handle_parsing_errors=True)
    result = agent_executor.invoke({
        'input': desired_format
    })
    print(result['output'])
    isSatisfied = str(input("Are you willing to continue with this response? Answer 'Y' for 'Yes' and 'N' for 'No' : ")).lower()
    if isSatisfied == 'y':
        flag = 0
    elif isSatisfied == 'n':
        flag = 1
    else:
        print("Invalid Input")



> Entering new AgentExecutor chain...
I need to remove all the citations and condense the text to fit into a Twitter post.
Action: remove_citations
Action Input: As the format of Inside No. 9 requires new characters each week, the writers were able to attract actors who may have been unwilling to commit to an entire series. In addition to Pemberton—who played Graham, the father—"Last Gasp" stars Sophie Thompson as Jan, the mother; Lucy Hutchinson as 9-year-old Tamsin; David Bedella as the pop star Frankie J Parsons; Tamsin Greig as Sally, of WishmakerUK; and Adam Deacon as Si, Parsons's assistant. Unusually for Inside No. 9, the episode does not feature Shearsmith in a credited acting role.As the format of Inside No. 9 requires new characters each week, the writers were able to attract actors who may have been unwilling to commit to an entire series. In addition to Pemberton—who played Graham, the father—"Last Gasp" stars Sophie Thompson as Jan, the mother; Lucy Hutchinson as 9-year-

In [63]:
print(result['output'])

As the format of Inside No. 9 requires new characters each week, the writers were able to attract actors who may have been unwilling to commit to an entire series. In addition to Pemberton—who played Graham, the father—"Last Gasp" stars Sophie Thompson as Jan, the mother; Lucy Hutchinson as 9-year-old Tamsin; David Bedella as the pop star Frankie J Parsons; Tamsin Greig as Sally, of WishmakerUK; and Adam Deacon as Si, Parsons's assistant. Unusually for Inside No. 9, the episode does not feature Shearsmith in a credited acting role. #InsideNo9 #TVSeries #Actors


<<< End Of Code >>>